# Experimental codes for testing potential arithematic precision errors.

The following results only take rtree into account, not including libspatialindex, since libspatialindex is not more directly used than rtree.

# 1. The dimension of index database should always be greater than 1. (0 or 1 dimension will fail).

In [41]:
from rtree import index

In [10]:
#points-to-point intersection test in 1 dimension
p = index.Property()
p.dimension = 1
p.dat_extension = 'data'
p.idx_extension = 'index'
db = index.Index(properties = p, interleaved = False)
db.insert(1, (0, 0),obj = ['a']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0.0, 0.0),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0.0, 0.00000000),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
hits = list(db.intersection((0, 0),objects = True))
[(item.object, item.bbox) for item in hits]

RTreeError: Error in "Index_Create": Spatial Index Error: IllegalArgumentException: initNew: Property Dimension must be greater than 1

# 2. The arithematic precision is 16 decimal digits, which comes from the double type's precision. ( The program will only takes the top 16 decimal digits of the number as itself, no matter how large or small this number is.)

For the top one, the retrieving will not consider they are the same one, since the last digit can be distinguished.
For the bottom one, the retrieving will consider they are the same, since the last digit can not be distinguished because 16 decimal digits is largest number they can capture.

In [6]:
# intervals-to-point intersection test in 4 dimension
print('----------------')
p = index.Property()
p.dimension = 4
p.dat_extension = 'data'
p.idx_extension = 'index'
db = index.Index(properties = p, interleaved = False)
# 16 decimal digits
db.insert(1, (0, 0, 0, 0, 0, 0, 9999999999999998,9999999999999998),obj = ['a']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 9999999999999998,9999999999999998),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 9999999999999998,9999999999999998),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
hits = list(db.intersection((0, 0, 0, 0, 0, 0, 9999999999999999,9999999999999999),objects = True))
[(item.object, item.bbox) for item in hits] 
print('----------------')
p = index.Property()
p.dimension = 4
p.dat_extension = 'data'
p.idx_extension = 'index'
# 17 decimal digits
db = index.Index(properties = p, interleaved = False)
db.insert(1, (0, 0, 0, 0, 0, 0, 99999999999999998,99999999999999998),obj = ['a']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 99999999999999998,99999999999999998),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 99999999999999998,99999999999999998),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
hits = list(db.intersection((0, 0, 0, 0, 0, 0, 99999999999999999,99999999999999999),objects = True))
[(item.object, item.bbox) for item in hits] 

----------------
----------------


[(['a'], [0.0, 0.0, 0.0, 1e+17, 0.0, 0.0, 0.0, 1e+17]),
 (['b'], [0.0, 0.0, 0.0, 1e+17, 0.0, 0.0, 0.0, 1e+17]),
 (['b'], [0.0, 0.0, 0.0, 1e+17, 0.0, 0.0, 0.0, 1e+17])]

# 3. The limitation of arithematic precision is 324 decimal digits, which is because from the python minimal measurement of distinguishing two different number.

In [18]:
((10**(-323) == 0.0),(10**(-324) == 0.0))

(False, True)

In [17]:
# intervals-to-point intersection test in 4 dimension
p = index.Property() 
p.dimension = 4
p.dat_extension = 'data'
p.idx_extension = 'index'
db = index.Index(properties = p, interleaved = False)
db.insert(1, (0, 0, 0, 0, 0, 0, 0-10**(-323), 0),obj = ['a']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 0-10**(-323), 0),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 0-10**(-324), 0),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
hits = list(db.intersection((0, 0, 0, 0, 0, 0, 0-10**(-323),0-10**(-324)),objects = True))
[(item.object, item.bbox) for item in hits]

[(['a'], [0.0, 0.0, 0.0, -1e-323, 0.0, 0.0, 0.0, -1e-323]),
 (['b'], [0.0, 0.0, 0.0, -1e-323, 0.0, 0.0, 0.0, -1e-323]),
 (['b'], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])]

# 4. The intersection for retrieving is closed intersection, in contrast with open intersection. (-0.000) is also considered as the same number as 0, but both representation will be displayed at the same time.

In [26]:
#points-to-point intersection test in 2 dimension
p = index.Property()
p.dimension = 3
p.dat_extension = 'data'
p.idx_extension = 'index'
db = index.Index(properties = p, interleaved = False)
db.insert(1, (0, 0,0,0, 0.000000, -0.0000),obj = ['a']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0.0, 0.0, 0,0, -0.000000, -0.0000),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0.0, 0.00000000, 0,0, -0.000000, -0.0000),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
hits = list(db.intersection((0, -0, 0,0, -0.000000, 0.0000),objects = True))
[(item.object, item.bbox) for item in hits]

[(['a'], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 (['b'], [0.0, 0.0, -0.0, 0.0, 0.0, -0.0]),
 (['b'], [0.0, 0.0, -0.0, 0.0, 0.0, -0.0])]

# 5. Round test. The round processing is not stable.

In [37]:
print('----------------')
p = index.Property()
p.dimension = 4
p.dat_extension = 'data'
p.idx_extension = 'index'
db = index.Index(properties = p, interleaved = False)
# 16 decimal digits
db.insert(1, (0, 0, 0, 0, 0, 0, 9999999999999991,9999999999999991),obj = ['a']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 9999999999999994,9999999999999994),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 9999999999999996,9999999999999996),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 9999999999999998,9999999999999998),obj = ['c']) #(x_min, y_min, z_min, x_max, y_max, z_max)
hits = list(db.intersection((0, 0, 0, 0, 0, 0, 9999999999999995,9999999999999995),objects = True))
[(item.object, item.bbox) for item in hits] 

----------------


[(['b'],
  [0.0, 0.0, 0.0, 9999999999999996.0, 0.0, 0.0, 0.0, 9999999999999996.0])]

In [38]:
print('----------------')
p = index.Property()
p.dimension = 4
p.dat_extension = 'data'
p.idx_extension = 'index'
db = index.Index(properties = p, interleaved = False)
# 16 decimal digits
db.insert(1, (0, 0, 0, 0, 0, 0, 9999999999999991,9999999999999991),obj = ['a']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 9999999999999994,9999999999999994),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 9999999999999996,9999999999999996),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 9999999999999998,9999999999999998),obj = ['c']) #(x_min, y_min, z_min, x_max, y_max, z_max)
hits = list(db.intersection((0, 0, 0, 0, 0, 0, 9999999999999993,9999999999999993),objects = True))
[(item.object, item.bbox) for item in hits] 

----------------


[(['a'],
  [0.0, 0.0, 0.0, 9999999999999992.0, 0.0, 0.0, 0.0, 9999999999999992.0])]

In [39]:
print('----------------')
p = index.Property()
p.dimension = 4
p.dat_extension = 'data'
p.idx_extension = 'index'
db = index.Index(properties = p, interleaved = False)
# 15 decimal digits
db.insert(1, (0, 0, 0, 0, 0, 0, 999999999999991,999999999999991),obj = ['a']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 999999999999994,999999999999994),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 999999999999996,999999999999996),obj = ['b']) #(x_min, y_min, z_min, x_max, y_max, z_max)
db.insert(2, (0, 0, 0, 0, 0, 0, 999999999999998,999999999999998),obj = ['c']) #(x_min, y_min, z_min, x_max, y_max, z_max)
hits = list(db.intersection((0, 0, 0, 0, 0, 0, 999999999999993,999999999999993),objects = True))
[(item.object, item.bbox) for item in hits] 

----------------


[]

The advantage of this library is that it is positive in the memory and storage, but not good at the precision control.